In [1]:
!pip install transformers

In [2]:
!git lfs install

Git LFS initialized.


In [3]:
!git clone https://huggingface.co/bert-base-uncased

fatal: destination path 'bert-base-uncased' already exists and is not an empty directory.


In [4]:
!pip install PyPDF2

In [5]:
!pip install pypdf

In [6]:
!pip install scipy

In [7]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [8]:
import pandas as pd
from pypdf import PdfReader
from nltk import pos_tag, sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
import re
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
import spacy

In [9]:
dataset = pd.read_csv('UpdatedResumeDataSet.csv')

In [10]:
dataset.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [11]:
nlp = spacy.load("en_core_web_sm")

In [12]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9., ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    sentences = sent_tokenize(text)
    features = {'feature': ""}
    stop_words = set(stopwords.words("english"))
    for sentence in sentences:
        if any(criteria in sentence for criteria in ['skills', 'education']):
            words = word_tokenize(sentence)
            words = [word for word in words if word not in stop_words]
            tagged_words = pos_tag(words)
            filtered_words = [word for word, tag in tagged_words if tag not in ['DT', 'IN', 'TO', 'PRP', 'WP']]
            features['feature'] += " ".join(filtered_words)
    return features


In [13]:
def get_embeddings(text, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    inputs = tokenizer(str(text), return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().to("cpu").numpy()
    return embeddings

In [14]:
dataset["Cleaned_Features"] = dataset["Resume"].apply(preprocess_text)

In [15]:
dataset.head(10)

,Category,Resume,Cleaned_Features
0,Data Science,Skills * Programming Languages: Python (pandas...,{'feature': 'skills programming languages pyth...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,{'feature': 'education details may may b e uit...
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",{'feature': 'areas interest deep learning cont...
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,{'feature': 'skills r python sap hana tableau ...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",{'feature': 'education details mca ymcaust far...
5,Data Science,"SKILLS C Basics, IOT, Python, MATLAB, Data Sci...",{'feature': 'skills c basics iot python matlab...
6,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,{'feature': 'skills python tableau data visual...
7,Data Science,Education Details \r\n B.Tech Rayat and Bahr...,{'feature': 'education details b tech rayat ba...
8,Data Science,Personal Skills â¢ Ability to quickly grasp t...,{'feature': 'personal skills ability quickly g...
9,Data Science,Expertise â Data and Quantitative Analysis â...,{'feature': 'expertise data quantitative analy...


In [16]:
dataset.drop(columns=['Resume'], inplace=True)

In [17]:
dataset.head()

,Category,Cleaned_Features
0,Data Science,{'feature': 'skills programming languages pyth...
1,Data Science,{'feature': 'education details may may b e uit...
2,Data Science,{'feature': 'areas interest deep learning cont...
3,Data Science,{'feature': 'skills r python sap hana tableau ...
4,Data Science,{'feature': 'education details mca ymcaust far...


In [18]:
jobs_dict = {
    "job_titles":["Data Scientist", "Machine Learning Engineer", "Data Analyst", "Backend Developer", "AI Research Intern", "Cloud DevOps Engineer", "Frontend React Developer", "Software Engineer (Full Stack)", "Product Data Scientist", "AI Prompt Engineer"],
    "job_descriptions":["We're seeking a Data Scientist proficient in Python, machine learning, and statistical modeling. Responsibilities include building predictive models, analyzing large datasets, and presenting insights. Experience with Pandas, Scikit-learn, and SQL required. 2+ years of experience preferred.",
                        "Looking for an ML Engineer with deep knowledge of TensorFlow or PyTorch. You'll build, optimize, and deploy ML models in production. Required: Python, APIs, cloud platforms (AWS or GCP). Strong software engineering fundamentals are a plus.",
                        "Join our team as a Data Analyst to transform raw data into actionable insights. Skills in Excel, SQL, Tableau, and Python are important. Understanding of business metrics and good communication is key.",
                        "Hiring a backend developer experienced in Node.js, REST APIs, and PostgreSQL. You'll design robust APIs and integrate with microservices. Knowledge of Docker and CI/CD pipelines is a bonus.",
                        "Assist in NLP and computer vision research. Looking for students or recent grads with experience in PyTorch, Hugging Face, and academic writing. Strong math/ML foundations are essential.",
                        "Manage and automate cloud infrastructure using AWS, Terraform, and Jenkins. Must be proficient in shell scripting and monitoring tools like Prometheus or Grafana.",
                        "React developer needed to build interactive web apps. Must know JavaScript, React, Redux, and responsive design. Familiarity with Figma or design tools is a plus.",
                        "Build and maintain full-stack applications using Python (Django or Flask) and React. Database knowledge (PostgreSQL/MySQL) and Git workflows required.",
                        "Work cross-functionally to analyze user behavior and experiment with product features. Tools: SQL, Python, A/B testing platforms. Experience with product analytics (Mixpanel, Amplitude) is beneficial.",
                        "Design and test prompts for large language models like GPT-4. Should understand NLP tasks, LLM APIs, and prompt engineering best practices. Creative writing or UX experience is a bonus."]
}

In [19]:
dataset_jobs = pd.DataFrame(jobs_dict)

In [20]:
dataset_jobs['Features'] =dataset_jobs['job_descriptions'].apply(lambda x : preprocess_text(x)['feature'])

In [21]:
dataset_jobs.head()

,job_titles,job_descriptions,Features
0,Data Scientist,We're seeking a Data Scientist proficient in P...,
1,Machine Learning Engineer,Looking for an ML Engineer with deep knowledge...,
2,Data Analyst,Join our team as a Data Analyst to transform r...,join team data analyst transform raw data acti...
3,Backend Developer,Hiring a backend developer experienced in Node...,
4,AI Research Intern,Assist in NLP and computer vision research. Lo...,


In [22]:
dataset_jobs.to_csv('dataset_jobs.csv')

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [24]:
# job_desc_embeddings = np.array([get_embeddings(desc, model_name) for desc in dataset_jobs['job_descriptions']]).squeeze()
# resume_embeddings = np.array([get_embeddings(text, model_name) for text in dataset['Cleaned_Features']]).squeeze()

In [25]:
pairs = []

for i, resume in tqdm(enumerate(dataset['Cleaned_Features'])):
    for j, text in enumerate( dataset_jobs['job_descriptions']):
        resume_emb = np.array(get_embeddings(resume, model_name)).squeeze()
        text_emb = np.array(get_embeddings(text, model_name)).squeeze()
        similarity = cosine_similarity([text_emb], [resume_emb])
        print(round(similarity[0][0] * 100, 2))
        match_score = round(similarity[0][0] * 100, 2)  # Scale to 0–100%
        pairs.append((resume, text, match_score))

0it [00:00, ?it/s]

66.83
67.22
63.07
61.51
62.73
60.22
64.46
63.1
66.71


1it [00:02,  2.43s/it]

63.91
71.26
72.76
67.86
66.51
72.21
64.92
70.86
64.41
72.43


2it [00:03,  1.79s/it]

...

62.36
62.45
68.41
71.52
66.59
65.95
71.45
63.77
70.05
63.64


961it [26:24,  1.52s/it]

69.76
69.77
66.79
70.11
63.71
66.04
64.39
65.66
68.99
66.52


962it [26:25,  1.65s/it]

67.66
69.25


In [26]:
dataset_similarity  = pd.DataFrame(pairs, columns=["Resume", "Job", "Match Score"])
dataset_similarity.to_csv('similarity_scores.csv', index=False)

In [27]:
dataset_similarity.head(20)

,Resume,Job,Match Score
0,{'feature': 'skills programming languages pyth...,We're seeking a Data Scientist proficient in P...,66.830002
1,{'feature': 'skills programming languages pyth...,Looking for an ML Engineer with deep knowledge...,67.220001
2,{'feature': 'skills programming languages pyth...,Join our team as a Data Analyst to transform r...,63.070000
3,{'feature': 'skills programming languages pyth...,Hiring a backend developer experienced in Node...,61.509998
4,{'feature': 'skills programming languages pyth...,Assist in NLP and computer vision research. Lo...,62.730000
5,{'feature': 'skills programming languages pyth...,Manage and automate cloud infrastructure using...,60.220001
6,{'feature': 'skills programming languages pyth...,React developer needed to build interactive we...,64.459999
7,{'feature': 'skills programming languages pyth...,Build and maintain full-stack applications usi...,63.099998
8,{'feature': 'skills programming languages pyth...,Work cross-functionally to analyze user behavi...,66.709999
9,{'feature': 'skills programming languages pyth...,Design and test prompts for large language mod...,63.910000


In [ ]:
dataset_similarity.drop(columns=['Match Score_rounded'], inplace=True)

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

In [30]:
job_desc_embeddings = np.array([get_embeddings(desc, model_name) for desc in dataset_similarity['Job']]).squeeze()
resume_embeddings = np.array([get_embeddings(text, model_name) for text in dataset_similarity['Resume']]).squeeze()

In [31]:
job_desc_embeddings.shape

(9620, 768)

In [32]:
resume_embeddings.shape

(9620, 768)

In [33]:
X = np.hstack((resume_embeddings, job_desc_embeddings))
y = dataset_similarity['Match Score'].values

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
regressor = GradientBoostingRegressor()
regressor.fit(X_train, y_train)

GradientBoostingRegressor()